### **Import Dependencies**

In [21]:
import os
from semantic_router import Route, RouteLayer
from semantic_router.encoders import OpenAIEncoder
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferWindowMemory
from IPython.display import display, Markdown
import pandas as pd

In [20]:
df = pd.read_csv("indian_business_criteria.csv")
df.head(5)

,State,Domain,Sub-Domain,Government Fees (INR),Documents Required,Eligibility Criteria,State-Specific Variation,Processing Time (Days)
0,Andhra Pradesh,Food,Restaurant,942,Health Clearance;ID Proof;Address Proof;Busine...,Age: 21+; Permanent Residence: Yes; Taxable In...,State residency required,10
1,Andhra Pradesh,Food,Catering,266,Safety Certificate;Business License,Age: 18+; Permanent Residence: No; Taxable Inc...,Additional certification required,7
2,Andhra Pradesh,Food,Street Food,474,Address Proof;Tax Registration;ID Proof,Age: 25+; Permanent Residence: No; Taxable Inc...,Additional certification required,17
3,Andhra Pradesh,Cloth,Boutique,712,Business License;Health Clearance;ID Proof,Age: 21+; Permanent Residence: Yes; Taxable In...,Additional certification required,26
4,Andhra Pradesh,Cloth,Online Store,587,Address Proof;Safety Certificate;Health Cleara...,Age: 25+; Permanent Residence: Yes; Taxable In...,No specific state variation,11


### **Load Data**

In [22]:
loader = CSVLoader(file_path='indian_business_criteria.csv')
data = loader.load()
print(f"Total number of new records loaded: {len(data)}")

Total number of new records loaded: 420


### **OpenAI**

In [24]:
os.environ["OPENAI_API_KEY"] = "API-KEY"
encoder = OpenAIEncoder()
embeddings = OpenAIEmbeddings()

### **LLM**

In [25]:
# Initialize the LLM
llm = ChatOpenAI(
    model='gpt-4-turbo',
    temperature=0.0,
)
# Create memory instance
memory1 = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=10,
    return_messages=True,
    output_key="output"
)

### **Vectorstore**

In [26]:
# Split the data into Chunks (can be omitted for small datasets but useful for larger ones)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [27]:
def initialize_vectorstore(persist_directory="./Chroma", documents=None, embeddings=None):
    if documents:
        vstore = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            persist_directory=persist_directory
        )
        print(f"Vector store initialized with {len(documents)} documents.")
    else:
        try:
            vstore = Chroma(persist_directory=persist_directory,
                            embedding_function=embeddings)
            print("Loaded existing vector store from disk.")
        except Exception as e:
            vstore = None
            print(f"Failed to load vector store from disk: {e}")
    return vstore

def initialize_retriever(vstore, llm):
    if vstore:
        retriever = vstore.as_retriever()
        rtvr = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
        )
        print("Retriever and RetrievalQA initialized.")
    else:
        retriever = None
        rtvr = None
        print("Vector store is not initialized. Retriever and RetrievalQA not created.")
    return retriever, rtvr


vstore = initialize_vectorstore(
    persist_directory="./Chroma", 
    documents=splits, 
    embeddings=embeddings
)
retriever, rtvr = initialize_retriever(vstore, llm)


# import tools

Vector store initialized with 420 documents.
Retriever and RetrievalQA initialized.


In [28]:
rtvr("what to do to start a resturant in andhrapradesh")

C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'what to do to start a resturant in andhrapradesh',
 'result': 'To start a restaurant in Andhra Pradesh, you need to follow these steps and meet the specific requirements:\n\n1. **Eligibility Criteria:**\n   - Age: You must be 21 years or older.\n   - Permanent Residence: You must be a permanent resident of Andhra Pradesh.\n   - Taxable Income: Your taxable income should be ₹60,000 or more.\n\n2. **Documents Required:**\n   - Health Clearance\n   - ID Proof\n   - Address Proof\n   - Business License\n\n3. **Government Fees:**\n   - You will need to pay a fee of ₹942.\n\n4. **Processing Time:**\n   - The processing time for your application is approximately 10 days.\n\nMake sure you have all the required documents and meet the eligibility criteria before applying. This will help ensure a smoother application process for starting your restaurant in Andhra Pradesh.'}

### **Semantic Routing**

In [30]:
chit_chat_route = Route(
    name="chit_chat",
    utterances=[
        "Good morning! I hope you have a fantastic day ahead.",
        "Thank you so much for your help yesterday; it made a big difference!",
        "Hello there! It's great to see you again.",
        "I really appreciate all the hard work you've put into this project.",
        "Hi! How have you been? It's been a while.",
        "Thank you for your prompt response; it was very helpful.",
        "Welcome to the team! We're excited to have you on board.",
        "I just wanted to say thank you for always being so supportive.",
        "Hey! Hope you're doing well. Just wanted to check in.",
        "Thanks for your patience and understanding throughout this process.",
        "Goodbye! Have a wonderful evening.",
        "See you later! Take care.",
        "Thank you again for everything. Goodbye for now!",
        "It was great catching up. Bye for now!",
        "Farewell! Wishing you all the best.",
        "Take care and see you soon!",
        "Goodbye! Let’s stay in touch.",
        "Thanks for your time. Have a good day!",
        "It’s been a pleasure working with you. Goodbye!",
        "Catch you later! Have a good one.",
        "Hey! What's new with you?",
        "Long time no see! How have you been?",
        "Good afternoon! How's your day going?",
        "Hey there! Any plans for the weekend?",
        "Just wanted to say hi! How are things?",
        "Hi! It's nice to hear from you.",
        "Good evening! How was your day?",
        "Thanks for reaching out! How can I assist you?",
        "Hope you're having a great day!",
        "Hi! What's up?",
        "Hello! How's everything?",
        "Hey! What's happening?",
        "Thanks for your help! Have a great day.",
        "Take it easy! Talk to you soon.",
        "Have a good one! See you around.",
        "It's been nice chatting. Take care!",
        "Stay well! Goodbye for now."
    ],
    keywords=[
        "hello", "hi", "hey", "good morning", "good afternoon", "good evening", "goodbye",
        "bye", "farewell", "see you", "take care", "thanks", "thank you", "appreciate", 
        "welcome", "great", "nice", "hope", "how", "been", "catch", "stay", "later", 
        "evening", "day", "time", "talk", "soon"
    ]
)

In [31]:
routes = [
    chit_chat_route
]

rl = RouteLayer(
    encoder=encoder,
    routes=routes
)


In [37]:
def chit_chat():
    return (
        "If the user greets you, politely greet them back. "
        "If the user thanks you, acknowledge their thanks politely."
    )

def general():
    return (
        "Please provide responses based solely on the context provided."
        "Do not include any additional information or context outside of what is given."
    )

### **Agent**

In [38]:
tools = [
    Tool(
        name="QA System",
        func=rtvr.run,
        description=(
            "Useful for answering questions about the process of starting a business in various cities, "
            "things to do, and more in specific cities. Specify the city and the query."
        ),
    ),
]

In [39]:
# Initialize Agent
agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    max_iterations=3,
    early_stopping_method="generate",
    memory=memory1,
)

In [40]:
# update the system prompt
system_message = """You are a business assistant, helping users in their initiative to setup new business.
Maintain a friendly and pleasant attitude at all times. Use positive language and avoid any criticism of the company or its services.
Pay close attention to the SYSTEM NOTES provided with user queries.
For every query the user makes, you MUST search the database first.
ONLY provide answers with the given context.
These notes contain useful information to enhance your responses and assist you with the user's specific needs and preferences.
"""
new_prompt = agent.agent.create_prompt(
    system_message=system_message, 
    tools=tools,
)
agent.agent.llm_chain.prompt = new_prompt

In [41]:
def test(prompt):
    route = rl(prompt)
    query = prompt
    
    if route.name == "chit_chat":
        query += f" (SYSTEM NOTE: {chit_chat()})"
        result = agent.invoke(query)['output']
        return result
    
    else:
        query += f" (SYSTEM NOTE: {general()})"
        result = agent.invoke(query)['output']
        return result

## **Testing**

In [42]:
responce = test("what to do to start a resturant in andhrapradesh")
display(Markdown(responce))

To start a restaurant in Andhra Pradesh, you need to meet the eligibility criteria, gather required documents such as health clearance, ID proof, address proof, and business license, pay the government fee of ₹942, and submit the application. The processing time is approximately 10 days. After approval, ensure compliance with local health and safety regulations.

In [43]:
responce = test("hi")
display(Markdown(responce))

Hello! How can I assist you today?

In [44]:
responce = test("what was my first question")
display(Markdown(responce))

Your first question was about what to do to start a restaurant in Andhra Pradesh.

In [45]:
responce = test("who is the current primemenester of india")
display(Markdown(responce))

I'm here to assist with business-related queries. Please let me know if you have any questions regarding starting or managing a business!

In [46]:
responce = test("the current primemenester of india is narendra modi")
display(Markdown(responce))

I'm here to assist with business-related queries. Please let me know if you have any questions regarding starting or managing a business!

In [47]:
responce = test("who is the current primemenester of india")
display(Markdown(responce))

I'm here to assist with business-related queries. Please let me know if you have any questions regarding starting or managing a business!